In [1]:
#!pip3 install pandas
#!pip3 install sklearn
#!pip3 install torch

In [9]:
import pandas as pd
import numpy as np
import pickle
import math
import os

In [10]:
objects = []
with (open("all_data.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [11]:
print(str(len(objects)))
pdata=objects[0]

1


In [12]:
pdata.shape

(168, 20)

#### Extract Catdef for ARIMA

In [13]:
extraction_dir="catdef_extraction"
os.system("mkdir -v "+extraction_dir)

mkdir: created directory 'catdef_extraction'


0

In [14]:
catdef_dims=pdata.apply(lambda row:row['catdef'].shape,axis=1).tolist()[0]


In [15]:
catdef_dims

(33, 37)

In [23]:
nan_counts_obs=set()
num_nan_dps=0
for row_idx in range(catdef_dims[0]):
    print(f"row is {row_idx} of {catdef_dims[0]} ...")
    for col_idx in range(catdef_dims[1]):
        the_header=f"catdef_row.{row_idx}.col.{col_idx}"
        csv_target=f"{extraction_dir}/catdef.{the_header}.csv"
        the_data=pdata.apply(lambda row:row['catdef'].item((row_idx,col_idx)),axis=1).tolist()
        nan_data=[d for d in the_data if np.isnan(d)]
        num_nans=len(nan_data)
        if(num_nans>0):
            nan_counts_obs.add(num_nans)
            num_nan_dps=num_nan_dps+1
            #this means that any nan data is never written!
            continue
        temp_df=pd.DataFrame.from_dict({the_header:the_data})
        temp_df.to_csv(csv_target,index=False)
        #break
    #break
print(f"nan_counts_obs {nan_counts_obs}")
print(f"num nan data points : {num_nan_dps}")
#this shows that any series is either all-data or all-nan

row is 0 of 33 ...
row is 1 of 33 ...
row is 2 of 33 ...
row is 3 of 33 ...
row is 4 of 33 ...
row is 5 of 33 ...
row is 6 of 33 ...
row is 7 of 33 ...
row is 8 of 33 ...
row is 9 of 33 ...
row is 10 of 33 ...
row is 11 of 33 ...
row is 12 of 33 ...
row is 13 of 33 ...
row is 14 of 33 ...
row is 15 of 33 ...
row is 16 of 33 ...
row is 17 of 33 ...
row is 18 of 33 ...
row is 19 of 33 ...
row is 20 of 33 ...
row is 21 of 33 ...
row is 22 of 33 ...
row is 23 of 33 ...
row is 24 of 33 ...
row is 25 of 33 ...
row is 26 of 33 ...
row is 27 of 33 ...
row is 28 of 33 ...
row is 29 of 33 ...
row is 30 of 33 ...
row is 31 of 33 ...
row is 32 of 33 ...
nan_counts_obs {168}
num nan data points : 363


### Execution

After creating the files, inside the docker container build with the R dockerfile, run the Rscript in a command like so : 

`
find catdef_extraction/*.csv|grep -Piv 'preds'|awk '{ print"./arima_train_cli.R " $0   }' |parallel -j 6 --eta
`

Change the "-j 6" for parallel to be perhaps "-j+0" to use all the cores? or the 6 to some higher or lower value depending on the number of cores.  The command can also be run inside a "screen" so as to help ensure that the program's execution is not interrupted.

.RData files are generated which contain the arima models and predictions for each of the 24 test periods as well as a prediction for the "25th" test period (Jan 2017).